## Justificatif Classifier

In [4]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential,load_model
from keras.layers import Conv2D, MaxPooling2D,BatchNormalization
from keras.layers import Activation, Dropout, Flatten, Dense
#Add Callbacks, e.g. ModelCheckpoints, earlystopping, csvlogger.
from keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing import image
from PIL import ImageEnhance, ImageFilter,Image
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os
import cv2
import sys
from keras.utils.vis_utils import plot_model

In [5]:
INPUT_SHAPE = (256, 256, 3)
# Define the number of classes
num_classes = 4

# Create a sequential model
model = Sequential()

# Add convolutional layers
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=INPUT_SHAPE))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the feature maps
model.add(Flatten())

# Add fully connected layers
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))

# Add the output layer
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# Print a summary of the model
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 127, 127, 32)     0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 62, 62, 64)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 60, 60, 128)       73856     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 30, 30, 128)     

In [6]:
# SIZE = 1500
# aspect_ratio = 1.5
# INPUT_SHAPE = (SIZE, int(SIZE*aspect_ratio), 3)
# # INPUT_SHAPE = (1240, 1240, 3)
# model = Sequential()
# model.add(Conv2D(32, (3, 3), input_shape=INPUT_SHAPE))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(BatchNormalization())

# model.add(Conv2D(32, (3, 3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(BatchNormalization())

# model.add(Conv2D(64, (3, 3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(BatchNormalization())

# model.add(Flatten())
# model.add(Dense(64))
# model.add(Activation('relu'))
# model.add(Dropout(0.5))
# model.add(Dense(4))
# model.add(Activation('softmax'))

# model.compile(loss='categorical_crossentropy',
#               optimizer='rmsprop',
#               metrics=['accuracy'])

# plot_model(model, to_file='model_architecture.png', show_shapes=True)

In [12]:
def straighten_image(pil_image):
    try:
        # cv_image = np.array(pil_image)
        cv_image = cv2.cvtColor(np.array(pil_image), cv2.COLOR_BGR2RGB)
        # Convert to gray
        gray = cv2.cvtColor(cv_image, cv2.COLOR_BGR2GRAY)

        # Threshold the image
        _, threshold_image = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY)

        # Perform morphological opening
        kernel = np.ones((5, 5), np.uint8)
        threshold_image = cv2.morphologyEx(threshold_image, cv2.MORPH_OPEN, kernel)

        # Perform morphological closing
        threshold_image = cv2.morphologyEx(threshold_image, cv2.MORPH_CLOSE, kernel)

        # Threshold the image again
        _, threshold_image = cv2.threshold(threshold_image, 128, 255, cv2.THRESH_BINARY_INV)
        # Find contours
        contours, hierarchy = cv2.findContours(threshold_image, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
        contours = sorted(contours, key=cv2.contourArea, reverse=True)[0:10]

        # Get the largest contour
        largest_contour = contours[1]
        image_area = cv_image.shape[0] * cv_image.shape[1]
        contour_area = cv2.contourArea(largest_contour)
        # Calculate the proportion of the contour area compared to the image area
        proportion = contour_area / image_area
        if proportion <= 0.3:
            return  pil_image

        # Contour detection
        # cv2.drawContours(cv_image, [largest_contour], -1, (0, 255, 0), 2)
        # Find the convex hull of the contour
        hull = cv2.convexHull(largest_contour)
        input_points = np.zeros((4, 2), dtype="float32")
        # Find the extreme points of the convex hull
        leftmost = tuple(hull[hull[:, :, 0].argmin()][0])
        rightmost = tuple(hull[hull[:, :, 0].argmax()][0])
        topmost = tuple(hull[hull[:, :, 1].argmin()][0])
        bottommost = tuple(hull[hull[:, :, 1].argmax()][0])
        bottom_right , top_right , top_left , bottom_left = None, None, None, None
        
        # Sort the extreme points
        extreme_points = [leftmost, rightmost, topmost, bottommost]
        sorted_points = sorted(extreme_points, key=lambda x: (x[0], x[1]))

        sorted_points = sorted(sorted_points, key=lambda p: p[0])
        left_points = sorted_points[:2]
        min_y_point = min(left_points, key=lambda p: p[1])
        top_left = min_y_point
        for point in left_points:
            if point != min_y_point:
                bottom_left = point
                break
    
        right_points = sorted(sorted_points, key=lambda p: p[0], reverse=True)[:2]
        top_right = min(right_points, key=lambda p: p[1])

        for point in right_points:
            if point != top_right:
                bottom_right = point
                break
        if bottom_right and top_right and top_left and bottom_left:
            rect = cv2.minAreaRect(largest_contour)
            # Top-left corner: (0, 0)
            # Top-right corner: (img.shape[1], 0)
            # Bottom-right corner: (img.shape[1], img.shape[0])
            # Bottom-left corner: (0, img.shape[0])
            # Define the target rectangle corners
            target_corners = np.array([[0, 0], [cv_image.shape[1], 0], [cv_image.shape[1], cv_image.shape[0]], [0, cv_image.shape[0]]], dtype=np.float32)

            # Define the source rectangle corners
            source_corners = np.array([top_left, top_right, bottom_right, bottom_left], dtype=np.float32)


            # new_source_corners = np.array([top_left, bottom_left, bottom_right, top_right], dtype=np.float32)

            transformation_matrix, _ = cv2.findHomography(source_corners, target_corners,cv2.RANSAC,5.0)
            straightened_invoice = cv2.warpPerspective(cv_image, transformation_matrix, (cv_image.shape[1], cv_image.shape[0]))
            pil_image = Image.fromarray(straightened_invoice)

            return pil_image
        else:
            # pil_image.show()

            return pil_image

    except Exception as e:
        print("An error occurred during image processing:", str(e))
        sys.exit()
    

In [8]:
# Define a function to perform image processing
def image_processing(image):
    try:
        # Convert image to PIL Image
        pil_image = Image.fromarray(np.uint8(image))
        # pil_image.thumbnail(target_size, Image.LANCZOS)

        # pil_image.show()
        pil_image = pil_image.convert('RGB')
        pil_image = straighten_image(pil_image)
        # print(pil_image.size)
        # Apply median filter
        filtered_image = pil_image.filter(ImageFilter.MedianFilter(size=1))
        # Apply sharpening
        sharpened_image = filtered_image.filter(ImageFilter.SHARPEN)
        # Enhance contrast
        enhancer = ImageEnhance.Contrast(sharpened_image)
        enhanced_image = enhancer.enhance(1.5)  # Increase contrast by a factor of 1.5
        # Enhance brightness
        enhancer = ImageEnhance.Brightness(enhanced_image)
        final_image = enhancer.enhance(1.2)
        # Apply Gaussian blur filter
        pil_image = final_image.filter(ImageFilter.GaussianBlur(radius=1))
        # pil_image.show()
        # Convert back to numpy array
        numpy_image = np.array(pil_image)
    
        # Scale pixel values to [0, 1]
        numpy_image = numpy_image / 255.0
    except Exception as e:
        print("An error occurred during image processing:", str(e))
        sys.exit()
    return numpy_image


In [10]:
batch_size = 32
num_epochs = 25
train_datagen = ImageDataGenerator(preprocessing_function = image_processing)
validation_datagen = ImageDataGenerator(preprocessing_function = image_processing)
test_datagen = ImageDataGenerator(preprocessing_function = image_processing)
SIZE = 256
aspect_ratio = 1
target_size = (SIZE,int(SIZE*aspect_ratio))
train_generator = train_datagen.flow_from_directory(
        'train3',  # this is the input directory
        batch_size=batch_size,
        shuffle=False,
        target_size=target_size,
        class_mode='categorical')  

# this is a similar generator, for validation data
validation_generator = validation_datagen.flow_from_directory(
        'validation3',
        batch_size=batch_size,
        shuffle=False,
        target_size=target_size,
        class_mode='categorical')

# this is a similar generator, for test data
test_generator = test_datagen.flow_from_directory(
        'test3',
        batch_size=batch_size,
        shuffle=False,
        target_size=target_size,
        class_mode='categorical')

Found 4857 images belonging to 4 classes.
Found 1938 images belonging to 4 classes.
Found 1950 images belonging to 4 classes.


In [13]:
try:
    history = model.fit_generator(
            train_generator,
            steps_per_epoch=train_generator.n//batch_size,
            epochs=num_epochs,
            validation_data=validation_generator,
            validation_steps=validation_generator.n//batch_size)

except Exception as e:
    print("An error occurred", str(e))
    sys.exit()
# Evaluate the model on the test set
    # test_loss, test_acc = model.evaluate_generator(test_generator, steps=test_generator.n//batch_size)
    # print('Test accuracy:', test_acc)

# Make predictions on the test set
    # test_predictions = model.predict_generator(test_generator, steps=test_generator.n//batch_size)
# Get the true labels and predicted labels for the test data


C:\Users\charbel.nasr\AppData\Local\Temp\ipykernel_24544\454017111.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/25
151/151 [==============================] - 689s 5s/step - loss: 1.3229 - accuracy: 0.5691 - val_loss: 0.7138 - val_accuracy: 0.7297
Epoch 2/25
151/151 [==============================] - 915s 6s/step - loss: 0.2906 - accuracy: 0.9115 - val_loss: 0.0860 - val_accuracy: 0.9781
Epoch 3/25
151/151 [==============================] - 783s 5s/step - loss: 0.4581 - accuracy: 0.9691 - val_loss: 0.0739 - val_accuracy: 0.9839
Epoch 4/25
151/151 [==============================] - 728s 5s/step - loss: 0.0158 - accuracy: 0.9934 - val_loss: 0.1650 - val_accuracy: 0.9839
Epoch 5/25
151/151 [==============================] - 679s 5s/step - loss: 0.1458 - accuracy: 0.9809 - val_loss: 0.0788 - val_accuracy: 0.9802
Epoch 6/25
151/151 [==============================] - 662s 4s/step - loss: 0.1206 - accuracy: 0.9890 - val_loss: 0.0984 - val_accuracy: 0.9844
Epoch 7/25
151/151 [==============================] - 662s 4s/step - loss: 0.0463 - accuracy: 0.9907 - val_loss: 0.2197 - val_accuracy: 0.9833

In [15]:
# Save your model
model.save('my_model_5.h5')

loaded_model = load_model('my_model_5.h5')


# Get the true labels and predicted labels for the test data
true_labels = test_generator.classes
predicted_labels = loaded_model.predict_generator(test_generator).argmax(axis=1)

# Compute the accuracy, precision, and recall
accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels, average='weighted')
recall = recall_score(true_labels, predicted_labels, average='weighted')
test_loss, test_acc = loaded_model.evaluate_generator(test_generator, steps=test_generator.n//batch_size)
print('Test accuracy:', test_acc)
# Print the results
print('Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)

# Print the ground truth and predicted labels for the first 10 images
# print("Ground Truth Labels: ")
# print([class_labels[np.argmax(label)] for label in test_labels[:10]])
# print("\nPredicted Labels: ")
# print([class_labels[np.argmax(pred)] for pred in test_predictions[:10]])
# fig, axs = plt.subplots(nrows=5, ncols=2, figsize=(10, 20))
# for i, ax in enumerate(axs.flatten()):
#     if i < len(test_images):
#         ax.imshow(test_images[i])
#         ax.axis('off')
#         ax.set_title(f"Ground Truth: {class_labels[np.argmax(test_labels[i])]}, Predicted: {class_labels[np.argmax(test_predictions[i])]}")
#     else:
#         ax.axis('off')
# plt.tight_layout()


C:\Users\charbel.nasr\AppData\Local\Temp\ipykernel_24544\3402059730.py:9: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  predicted_labels = loaded_model.predict_generator(test_generator).argmax(axis=1)
C:\Users\charbel.nasr\AppData\Local\Temp\ipykernel_24544\3402059730.py:15: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  test_loss, test_acc = loaded_model.evaluate_generator(test_generator, steps=test_generator.n//batch_size)


Test accuracy: 0.9885416626930237
Accuracy: 0.9882051282051282
Precision: 0.9895040421297358
Recall: 0.9882051282051282
